In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
from io import BytesIO
import os

# Ustawienia wykresów
sns.set(style="whitegrid")

# Ścieżka do archiwum ZIP
zip_path = "../data/raw/btc-price-history.zip"

# Wczytanie danych z pliku ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    # znajdź wszystkie pliki .csv w archiwum
    csv_files = [name for name in z.namelist() if name.endswith(".csv")]
    if not csv_files:
        raise FileNotFoundError("W archiwum ZIP nie znaleziono żadnego pliku .csv")
    
    # otwórz pierwszy znaleziony plik
    with z.open(csv_files[0]) as f:
        df = pd.read_csv(f)

# ---- WSTĘPNA ANALIZA ----
print("✅ Wczytano dane:", df.shape)
print(df.info())

# Konwersja Timestamp na datetime (jeśli występuje)
if "Timestamp" in df.columns:
    try:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="s", errors="coerce")
    except:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# Statystyki opisowe
display(df.describe(include="all"))

# ---- TWORZENIE FOLDERU NA WYKRESY ----
os.makedirs("docs", exist_ok=True)

# ---- WYKRESY EDA ----

# 1️⃣ Korelacja zmiennych numerycznych
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korelacja między zmiennymi numerycznymi")
plt.tight_layout()
plt.savefig("docs/heatmap_correlation.png")
plt.show()

# 2️⃣ Histogramy rozkładów dla kolumn numerycznych
for col in df.select_dtypes(include="number").columns:
    plt.figure(figsize=(8, 5))
    sns.histplot(df[col], kde=True, log_scale=(True if df[col].max() > 1000 else False))
    plt.title(f"Rozkład {col}")
    plt.tight_layout()
    plt.savefig(f"docs/hist_{col}.png")
    plt.close()

print("📊 Histogramy zapisane w katalogu 'docs/'")

# 3️⃣ Trend ceny w czasie
if "Timestamp" in df.columns and "Close" in df.columns:
    plt.figure(figsize=(12, 5))
    plt.plot(df["Timestamp"], df["Close"], linewidth=1)
    plt.title("Cena zamknięcia w czasie (Close)")
    plt.xlabel("Data")
    plt.ylabel("Cena [USD]")
    plt.tight_layout()
    plt.savefig("docs/close_trend.png")
    plt.show()
    print("📈 Wykres trendu ceny zapisany jako 'docs/close_trend.png'")
else:
    print("⚠️ Brak kolumn 'Timestamp' lub 'Close' — pominięto wykres trendu.")
